# EMNIST CNN Training Pipeline
This notebook trains a Convolutional Neural Network (CNN) on the EMNIST (Extended MNIST) dataset. Unlike standard MNIST which only contains digits, EMNIST contains both digits and handwritten letters.

Dataset Split: We use the 'balanced' split, which maps to 47 classes:
10 Digits (0-9)
26 Capital Letters (A-Z)
11 Lowercase Letters (a, b, d, e, f, g, h, n, q, r, t) - Note: Other lowercase letters are merged with uppercase because they look identical in handwriting (e.g., c, o, p, s).

In [1]:
#pip install tensorflow emnist matplotlib numpy

# Imports and Data Loading:

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from emnist import extract_training_samples, extract_test_samples
import numpy as np
import matplotlib.pyplot as plt

# 1. Load EMNIST 'balanced' dataset
# This might take a minute to download the first time
print("Downloading and loading EMNIST data...")
train_images, train_labels = extract_training_samples('balanced')
test_images, test_labels = extract_test_samples('balanced')

print(f"Training data shape: {train_images.shape}")
print(f"Test data shape: {test_images.shape}")
print(f"Unique classes: {len(np.unique(train_labels))}")

# 2. Define the Mapping (Class ID -> Character)
# This helps us visualize what we are training on
emnist_labels = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J',
    20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T',
    30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z',
    36: 'a', 37: 'b', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'n', 44: 'q', 45: 'r', 46: 't'
}

# Data Preprocessing and Orientation Fix:

Critical Note: EMNIST images from the python library are often rotated and flipped compared to normal images. We fix this here.

In [ ]:
# 3. Data Preprocessing

# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Reshape for CNN input: (Height, Width, Channel) -> (28, 28, 1)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

# Visualize a few samples to check orientation
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    # If images look sideways/flipped, we might need np.transpose, but usually
    # the 'emnist' library handles extraction well. Let's verify visually.
    plt.imshow(train_images[i].reshape(28,28), cmap='gray')
    label_id = train_labels[i]
    plt.title(f"{emnist_labels[label_id]}")
    plt.axis('off')
plt.show()

# Categorical conversion is not strictly needed if we use 'sparse_categorical_crossentropy'
print("Data ready for training.")

# Model Architecture
We use a standard CNN architecture:
    1. Conv2D: Extracts features (edges, curves).
    2. MaxPooling: Reduces dimensionality.
    3. Dropout: Prevents overfitting (memorizing the data).
    4. Dense (Output): 47 neurons with Softmax activation for probability distribution.

In [ ]:
# 4. Build the CNN Model
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    
    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Third Convolutional Block (Optional but good for accuracy)
    layers.Conv2D(128, (3, 3), activation='relu'),
    # layers.MaxPooling2D((2, 2)), # Might be too small to pool again
    
    layers.Flatten(),
    
    # Fully Connected Layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), # Drops 50% of neurons randomly during training to improve generalization
    
    # Output Layer (47 classes for EMNIST Balanced)
    layers.Dense(47, activation='softmax')
])

model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
We will train for about 5-10 epochs. Since the dataset is large, this should be sufficient to reach >85% accuracy.

In [ ]:
# 5. Train the Model
history = model.fit(
    train_images, 
    train_labels, 
    epochs=10, 
    batch_size=64, 
    validation_split=0.1 # Use 10% of training data to validate during training
)

# Evaluation and Saving:

In [ ]:
# 6. Evaluate on Test Data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"\nTest accuracy: {test_acc*100:.2f}%")

# 7. Plot Training History
plt.figure(figsize=(12, 4))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.show()

# 8. Save the Model
model.save('emnist_model.h5')
print("Model saved as 'emnist_model.h5'. You can now use this in the inference notebook.")

# Test Prediction

In [ ]:
# 9. Test with a random image from the test set
import random

idx = random.randint(0, len(test_images))
img = test_images[idx]
true_label = test_labels[idx]

# Reshape for prediction (1, 28, 28, 1)
input_img = img.reshape(1, 28, 28, 1)

prediction = model.predict(input_img)
predicted_label = np.argmax(prediction)

plt.figure(figsize=(2, 2))
plt.imshow(img.reshape(28, 28), cmap='gray')
plt.title(f"True: {emnist_labels[true_label]} | Pred: {emnist_labels[predicted_label]}")
plt.axis('off')
plt.show()